In [1]:
%pylab inline
import tensorflow as tf
from itertools import groupby
import pandas as pd
from collections import OrderedDict
from datetime import date
import pyodbc

Populating the interactive namespace from numpy and matplotlib


C:\Users\Vivi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Preprocessing (can be skipped after done once)

In [ ]:
records=pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/survstat/csv/cam_5y.csv',encoding='latin-1')
CoNew = pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/CoordBins.csv',encoding='latin-1')

In [ ]:
recordsX=records[['county','week','count','incidence']]
gb = recordsX.groupby(['week','county']).sum()

In [ ]:
gb.index[0]

In [ ]:
newDF = pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
x = 0
for i in gb.index:
    week = i[0]
    counties = i[1]
    counts = gb.values[x][0]
    incidences = gb.values[x][1]
    loc = CoNew[CoNew['county']==counties]
    if loc['X'].values>0:
        newDF=newDF.append({'week': str(week)[7:],'year':str(week)[:4],'county':str(counties),'count':str(counts),'incidence':str(incidences),'X':loc['X'].values,'Y':loc['Y'].values,'BinX':loc['BinX'].values,'BinY':loc['BinY'].values,'LocCounty':loc['county'].values}, ignore_index=True)    
    try:
        with open('StomachVirusInput.csv', 'a') as f:
            newDF.to_csv(f, header=False)
    except:
        pass
    newDF=pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
    x = x+1
#newDF.to_csv('StomachVirusInput.csv')

In [ ]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF.columns = ['ID','week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty']#.set_index('Unnamed: 0')
newDF = newDF.dropna()
newDF = newDF[['week','year','county','count','incidence','X','Y','BinX','BinY']].reset_index(drop=True)
newDF.to_csv('StomachVirusNew.csv')

In [ ]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF

In [ ]:
test = newDF[newDF['year']==2001][newDF[newDF['year']==2001]['week']==2]['incidence']

In [ ]:
def FindLabels(data):
    LabelDF = pd.DataFrame(columns=['count','incidence','county'])
    for w in data.index:
        try:
            thisyear = data.iloc[w]['year']
            thisweek = data.iloc[w]['week']
            thisarea = data.iloc[w]['county']
            if int(thisweek)<52:
                nextweek = int(thisweek)+1
                nextyear = int(thisyear)
            else:
                nextweek = 1
                nextyear = int(thisyear)+1
            nextdf = data[data['year']==nextyear][data[data['year']==nextyear]['week']==nextweek]
            nextcount = np.sum(nextdf[nextdf['county']==thisarea]['count'])
            if nextcount>0:
                nextincidence = np.sum(nextdf[nextdf['county']==thisarea]['incidence'])
                nextcounty = nextdf[nextdf['county']==thisarea]['county'].values[0]
            else:
                nextincidence=0  
                nextcounty = 'no Data'
        except:
            nextcount=0
            nextincidence=0  
            nextcounty = 'no Data'
        LabelDF=LabelDF.append({'count':nextcount,'incidence':nextincidence,'county':nextcounty}, ignore_index=True)
        if (w % 1000 == 0):
            print(w)
    return LabelDF
        
label = FindLabels(newDF)
label.to_csv('StomachOutput.csv')

# Load Processed Data

In [2]:
Labels=pd.read_csv('StomachOutput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data=pd.read_csv('StomachVirusInput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data = Data[['week','year','count','incidence','X','Y','county','BinX','BinY']]
dfboth = Data.join(Labels,lsuffix='x',how='outer').dropna()
Data = dfboth[['week','year','countx','incidencex','X','Y','countyx','BinX','BinY']].rename(index=str, columns={"countx": "count", "incidencex": "incidence"})
Labels = dfboth[['count','incidence']]

In [3]:
allMaps = []
for m in range(15):
    for n in range(52):
        year = 2001+m
        thisweek = Data[Data['year']==year][Data[Data['year']==year]['week']==n]
        thisweek = thisweek.reset_index(drop=True)
        currentMap = np.zeros((16,16))
        currentCount = np.zeros((16,16))
        for i in range(shape(thisweek)[0]):
            inc = thisweek['incidence'][i]
            x = int(thisweek['BinX'][i][1:-1])
            y = int(thisweek['BinY'][i][1:-1])
            currentMap[x][y]=currentMap[x][y]+inc
            currentCount[x][y]=currentCount[x][y]+1
        currentMap = currentMap/currentCount
        allMaps.append(np.nan_to_num(currentMap))

C:\Users\Vivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


## Normalize Data

In [ ]:
import sklearn
from sklearn.preprocessing import normalize
normalized = newDF
"""x = normalized['BinX']
normBinX =sklearn.preprocessing.normalize([x],axis=1)
normalized['BinX']=normBinX[0]
x = normalized['BinY']
normBinY =sklearn.preprocessing.normalize([x])
normalized['BinY']=normBinY[0]
x = normalized['X']
normX =sklearn.preprocessing.normalize([x])
normalized['X']=normX[0]
x = normalized['Y']
normY =sklearn.preprocessing.normalize([x])
normalized['Y']=normY[0]"""
x = normalized['count']
normcount =sklearn.preprocessing.normalize([x])
normalized['count']=normcount[0]
x = normalized['incidence']
normInc =sklearn.preprocessing.normalize([x])
normalized['incidence']=normInc[0]

In [ ]:
normLabels = Labels
x = normLabels['count']
normcount =sklearn.preprocessing.normalize([x])
normLabels['count']=normcount[0]
x = normLabels['incidence']
normInc =sklearn.preprocessing.normalize([x])
normLabels['incidence']=normcount[0]

In [25]:
lenall = Data.shape[0]
trainSize = lenall//6*5#define sizes for data sets (train: 2/3, val:1/6, test:1/6 of whole dataset) 
valSize = 0#lenall//6

#Split data into train, val and test set------------------------------------------
trainingData = Data.iloc[0:trainSize].reset_index(drop=True)
trainingLabels = Labels.iloc[0:trainSize].reset_index(drop=True)
validationData = Data.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
validationLabels = Labels.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
testData = Data.iloc[trainSize+valSize+1:].reset_index(drop=True)
testLabels = Labels.iloc[trainSize+valSize+1:].reset_index(drop=True)

In [26]:
#Method to get batches------------------------------------------------------------
def getBatches(batchsize,typeBatch):
    if typeBatch=="train":
        numBatches = shape(trainingData)[0]//batchsize
        Batches=[]
        for count in range(numBatches-1):
            batch = trainingData[count*batchsize:(count+1)*batchsize]
            labels = trainingLabels[count*batchsize:(count+1)*batchsize]
            b,l=formatBatch(batch,labels,batchsize)
            newb = addGlobalCNN(b,batch.reset_index(),batchsize,batch['week'].reset_index(),batch['year'].reset_index())
            #print(newb)
            Batches.append([newb,l])
    elif typeBatch=="val":
        batch = validationData.sample(batchsize)
        labels = validationLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    elif typeBatch=="test":
        batch = testData.sample(batchsize)
        labels = testLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    else:
        print("typeBatch needs to be set to one of these values: train, val, test.")
        pass
    return Batches
def getLastWeek(thisweek,thisyear):
    weekdata = Data[Data['year']==thisyear][Data[Data['year']==thisyear]['week']==thisweek]
    #print(weekdata)
    size = shape(weekdata)[0]
    batch = formatBatch(weekdata,weekdata,size)
    return batch,size,weekdata['countyx']

def formatBatch(batch,labels,batchsize):
    batch = batch.reset_index(drop=True)
    labels = labels.reset_index(drop=True)
    newl = []
    newb = []
    for i,b in enumerate(batch['week']):
        x = batch['X'][i][1:-1]
        y = batch['Y'][i][1:-1]
        try:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(x)/100,float(y)/100])
        except:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(0)/100,float(0)/100])
    for ind,l in enumerate(labels['count']):
        newl = np.append(newl,[l/10,(labels['incidence'][ind])/100],axis=0)
    return np.reshape(newb,(batchsize,5)),np.reshape(newl,(batchsize,2))
def addGlobalCNN(batch,dfBatch,batchsize,week,year):
    records = []
    newb = []
    for i in range(batchsize):
        thisweek = week['week'][i]
        thisyear = year['year'][i]
        MapNum = (thisyear-2001)*52+thisweek
        x = dfBatch['BinX'][i]
        y = dfBatch['BinY'][i]
        binrisk = allMaps[MapNum][int(x[1:-1])][int(y[1:-1])]
        newb = np.append(batch[i],binrisk/100)
        records.append(newb)
    return records

In [27]:
b = getBatches(50,'train')
#print(b)
for e in b:
    print("batch")
    print(e[0])
    print("labels")
    print(e[1])

batch
[array([1.        , 0.1       , 0.198928  , 0.07727029, 0.50739469,
       0.14363708]), array([1.        , 0.1       , 0.34366229, 0.08032911, 0.53221477,
       0.34366229]), array([1.        , 0.1       , 0.18432613, 0.12174144, 0.51778919,
       0.18432613]), array([1.        , 0.5       , 1.0144087 , 0.08098917, 0.49439435,
       0.62048158]), array([1.        , 0.1       , 0.24270536, 0.13723979, 0.52826784,
       0.24270536]), array([1.        , 0.2       , 0.21857354, 0.14209969, 0.51314749,
       0.28657427]), array([1.        , 0.1       , 0.22757731, 0.06947075, 0.49928801,
       0.31095818]), array([1.        , 0.2       , 0.37901348, 0.09738413, 0.48121566,
       0.30653881]), array([1.        , 0.1       , 0.39433904, 0.0731217 , 0.49713735,
       0.31095818]), array([1.        , 0.1       , 0.2489661 , 0.06410603, 0.50076556,
       1.21568431]), array([1.        , 0.1       , 0.13682698, 0.07926843, 0.47932012,
       0.22388555]), array([1.        , 0.1   

labels
[[0.7        0.32370743]
 [0.3        0.21395441]
 [0.5        0.61510805]
 [0.7        0.96193531]
 [0.6        0.50401887]
 [0.2        0.20260916]
 [1.         0.64821109]
 [0.6        1.57294067]
 [1.5        0.85391245]
 [0.4        0.77808297]
 [0.4        0.48784815]
 [0.6        0.7855679 ]
 [0.         0.        ]
 [0.4        1.00034544]
 [0.2        0.43355173]
 [0.1        0.17405094]
 [0.         0.        ]
 [0.7        1.17329035]
 [0.3        0.55744472]
 [0.4        0.88727905]
 [0.4        0.55302239]
 [0.1        0.29793432]
 [0.2        0.43803254]
 [1.         1.26242973]
 [0.3        0.70726225]
 [0.3        0.34346045]
 [0.         0.        ]
 [0.         0.        ]
 [0.3        0.2769024 ]
 [0.3        0.71541041]
 [0.8        2.09202197]
 [0.9        0.70602903]
 [1.1        1.76662231]
 [0.5        0.63141587]
 [0.3        0.79572888]
 [0.1        0.43783902]
 [0.2        0.4886301 ]
 [0.2        0.42843346]
 [0.2        0.49630313]
 [0.7        0.923

 [0.2        1.0578417 ]]
batch
[array([6.        , 0.5       , 0.20695653, 0.0864314 , 0.50121407,
       0.2830058 ]), array([6.        , 0.1       , 0.1185802 , 0.07812223, 0.47989734,
       0.28175506]), array([6.        , 0.7       , 0.90342461, 0.07066821, 0.51561465,
       0.51786165]), array([6.        , 0.3       , 1.16837971, 0.12087031, 0.50888766,
       2.16853717]), array([6.        , 0.1       , 0.13538313, 0.07485513, 0.51341046,
       0.27692604]), array([6.        , 0.3       , 0.42269131, 0.11973342, 0.51477755,
       0.58583119]), array([6.        , 4.5       , 0.85335672, 0.10237936, 0.53497897,
       0.73340231]), array([6.        , 0.1       , 0.15264198, 0.07816168, 0.51675196,
       0.58393898]), array([6.        , 0.2       , 0.4468619 , 0.09183153, 0.49153119,
       0.37098949]), array([6.        , 0.2       , 0.58256096, 0.11395613, 0.48757667,
       0.69224286]), array([6.        , 0.2       , 0.39551435, 0.11579704, 0.50919251,
       0.44669229]),

        2.75248961])]
labels
[[0.1        0.41044351]
 [0.2        0.44145454]
 [0.5        0.82069299]
 [0.7        2.37320593]
 [0.4        0.65995415]
 [0.1        0.31884931]
 [0.5        0.87632347]
 [0.1        0.40646735]
 [0.1        0.1374413 ]
 [0.9        1.09283861]
 [0.3        0.35440937]
 [0.3        0.75025585]
 [0.4        0.49811018]
 [0.         0.        ]
 [0.3        0.9826483 ]
 [0.7        1.21065555]
 [0.9        0.73760925]
 [0.4        1.18788877]
 [1.2        0.86382429]
 [0.1        0.42188159]
 [0.2        0.3188909 ]
 [0.1        0.36483035]
 [0.6        0.87071922]
 [0.1        0.66912011]
 [0.4        1.20245711]
 [0.3        1.11825255]
 [0.5        0.95162638]
 [0.2        0.80111103]
 [0.1        0.17569718]
 [0.3        0.63959978]
 [0.         0.        ]
 [0.9        1.36274808]
 [0.1        0.20818635]
 [1.1        0.95826409]
 [0.3        1.18666009]
 [0.1        0.25467253]
 [0.2        0.51317964]
 [0.1        0.29804285]
 [0.6        0.554412

labels
[[0.4        0.49667363]
 [0.4        0.53033794]
 [0.1        0.16543661]
 [0.         0.        ]
 [0.3        1.15763254]
 [0.7        1.21054048]
 [0.2        0.98866869]
 [0.1        0.16210083]
 [0.1        0.11745591]
 [0.1        0.12550201]
 [0.2        0.3901852 ]
 [0.2        0.36211746]
 [0.3        1.1767291 ]
 [0.3        0.54803034]
 [0.         0.        ]
 [0.2        0.25708816]
 [0.1        0.30914556]
 [0.4        0.54403286]
 [0.         0.        ]
 [0.         0.        ]
 [0.1        0.19697533]
 [0.3        0.61130161]
 [0.4        0.36206703]
 [0.2        0.6278614 ]
 [0.2        0.53367893]
 [0.5        0.39185762]
 [0.         0.        ]
 [0.6        0.86620431]
 [0.6        0.90827377]
 [0.1        0.52534804]
 [0.2        0.72104007]
 [0.         0.        ]
 [0.4        1.01649284]
 [0.5        0.96792741]
 [0.         0.        ]
 [0.         0.        ]
 [0.4        0.59714019]
 [0.         0.        ]
 [0.2        0.62262032]
 [0.3        0.246

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [36]:
tf.reset_default_graph()
%matplotlib notebook
learning_rate = 0.05
batch_size = 20
training_epochs = 100
display_step = 1
complexInput=False
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 6 
n_classes = 2 
step = 1

# tf Graph input
X = tf.placeholder("float32", [None,n_input])
Y = tf.placeholder("float32",[None,n_classes])

keep_probability = tf.placeholder("float")

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=1/n_input)),#**(-1/2))),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=1/n_hidden_1)),#**(-1/2))),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=1/n_hidden_2))#**(-1/2)))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'bout': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(x, weights['h1']), biases['b1'])),keep_probability)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])),keep_probability)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['bout']
    return out_layer

outP = multilayer_perceptron(X)

loss_op = tf.reduce_mean(tf.losses.mean_squared_error(Y,outP))
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

tf.summary.scalar("loss", loss_op)
tf.summary.scalar("sample output",(outP[0][0])*10)
tf.summary.scalar("sample target output",(Y[0][1])*100)

merged_summaries = tf.summary.merge_all()

#train_writer = tf.summary.FileWriter("./summaries/trainFFNAreaRisk", tf.get_default_graph())
#validation_writer = tf.summary.FileWriter("./summaries/validationFFNAreaRisk")

INFO:tensorflow:Summary name sample output is illegal; using sample_output instead.
INFO:tensorflow:Summary name sample target output is illegal; using sample_target_output instead.


In [ ]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    samplePred = []
    sample = []
    allLoss = []
    for n in range(training_epochs):
        losses = []
        for b in getBatches(batch_size,"train"):
            _,loss,output,_summaries = sess.run([train_op,loss_op,outP,merged_summaries], feed_dict={X: b[0],Y: b[1],keep_probability:kp})  
            lastout = output[1]
            losses = np.append(losses,loss)
            step = step + 1
            #train_writer.add_summary(_summaries, step)
            #if (step%1000==0):
             #   saver.save(sess,"./log/FFNCountyRisk.ckpt")
        #b = getBatches(batch_size,'val')
        #_,Valloss,output,_Valsummaries = sess.run([train_op,loss_op,outP,merged_summaries], feed_dict={X: np.reshape(b[0],(batch_size,5)),Y: np.reshape(b[1],(batch_size,2)),keep_probability:kp})  
        #print("validation Loss: ",Valloss)
        #validation_writer.add_summary(_Valsummaries, step)
        samplePred = np.append(samplePred,lastout[1])
        sample = np.append(sample,b[1][1][1])
        allLoss = np.append(allLoss,np.mean(losses))
        print("epoch: ",n,' - Step: ',step)
        print("pred: ",lastout)
        print("true: ",b[1][1])
        print("loss: ",np.mean(losses))
        #print(losses[0])

In [ ]:
allLoss

# Make New Predictions:

In [ ]:
thisweek = 52
thisyear = 2016
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    saver.restore(sess, "C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/log/FFNCountyRisk.ckpt")
    print("model loaded")
    #b,s,c = getLastWeek(thisweek,thisyear)
    batch_size = s
    _,loss,output= sess.run([train_op,loss_op,outP], feed_dict={X: newinput,Y: b[1],keep_probability:kp}) 
    print(output)


In [ ]:
predicted = pd.DataFrame(np.multiply(output,100))
counties = pd.DataFrame(c)

d = predicted.join(counties.reset_index(),lsuffix='x')
#d = d[['countyx','1']]
d.columns = ['count','incidence','ID','county']
d['week']=1#thisweek+1
d['year']=thisyear+1
d = d[['count','incidence','county','week','year']]
d.to_csv('Prediction_1_2017.csv')

In [ ]:
mses = []
for b in getBatches(batch_size,'train'):
    A = [b[0][0][1],b[0][0][2]]
    B = b[1][0]
    mse = ((A-B)**2).mean()
    mses.append(mse)